### Importing Libraries

In [16]:
from sklearn.feature_extraction.text import  TfidfVectorizer
from nltk.tokenize import wordpunct_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import re

### Pre_process text  and Generate Dataset

In [17]:
def pre_process(text):
    clean_text = re.sub(r'[*|?|!|\'|"|#|.|,|)|(|\|/|”|“|_|:|;]', r' ', text.lower())
    clean_text = clean_text.replace('\n', ' ')
    clean_text = clean_text.strip()
    clean_text = ' '.join(clean_text.split())
    return clean_text

In [18]:
corp = open('dataset/corpus.txt', 'r',encoding="utf8").read() 

In [19]:
pat = '\n{5,}\s+THE SECRET CACHE\n{5,}.*'
corp = re.search(pat, corp, flags=re.S+re.M).group()
corp = re.sub(r'-', r'', corp)
corp = pre_process(corp)
corp_toks = wordpunct_tokenize(corp)

In [20]:
labels = ['am', 'are', 'were', 'was', 'is', 'been', 'being', 'be']

In [21]:
def generate_inp(corp_toks, ksize):
    data = []
    for i, tok in enumerate(corp_toks):
        if tok in labels:
            inp_toks = corp_toks[max(i-ksize, 0):i] + corp_toks[i+1:min(i+ksize, len(corp_toks))]
            txt = ' '.join(inp_toks)
            data.append(txt)
    
    return data


In [22]:
def generate_data(corp_toks, ksize=5):
    corp_nolabel = [tok for tok in corp_toks if tok in labels]
    data = generate_inp(corp_toks, ksize)
    
    return data, corp_nolabel


In [23]:
def generate_test(corp_toks, ksize=5):
    data = []
    for i, tok in enumerate(corp_toks):
        if tok == '----':
            inp_toks = corp_toks[max(i-ksize, 0):i] + corp_toks[i+1:min(i+ksize, len(corp_toks))]
            txt = ' '.join(inp_toks)
            data.append(txt)
    
    return data

### Creating Pipeline for Regression 

In [1]:
X, y = generate_data(corp_toks)

pipe = Pipeline([
    ('tf', TfidfVectorizer(ngram_range=(1, 2), strip_accents='unicode')),
    ('clf', LogisticRegression(class_weight='balanced', max_iter=50))
])

pipe.fit(X, y)

NameError: name 'generate_data' is not defined

### Testing the Dataset

In [26]:
N = input()
test_corp = input()
test_corp = pre_process(test_corp)
test_toks = wordpunct_tokenize(test_corp)
test_txts = generate_test(test_toks)
test_preds = pipe.predict(test_txts)

for p in test_preds:
    print(p)

6
When the modern Olympics began in 1896, the initiators and organizers ---- looking for a great popularizing event, recalling the ancient glory of Greece. The idea of a marathon race came from Michel Breal, who wanted the event to feature in the first modern Olympic Games in 1896 in Athens. This idea was heavily supported by Pierre de Coubertin, the founder of the modern Olympics, as well as by the Greeks. The Greeks staged a selection race for the Olympic marathon on 10 March 1896 that ---- won by Charilaos Vasilakos in 3 hours and 18 minutes (with the future winner of the introductory Olympic Games marathon coming in fifth). The winner of the first Olympic Marathon, on 10 April 1896 (a male-only race), was Spyridon "Spyros" Louis, a Greek water-carrier, in 2 hours 58 minutes and 50 seconds. The women's marathon ---- introduced at the 1984 Summer Olympics (Los Angeles, USA) and ---- won by Joan Benoit of the United States with a time of 2 hours 24 minutes and 52 seconds. Since the mo